# EEG - Flow

## 4. ICA select
Saves the ICA with the reviewer in the filename

Last edit: 07.05.2023 03:41
@anguyen

In [ ]:
import os
from datetime import datetime
import matplotlib.pyplot as plt

from mne.viz import set_browser_backend

%matplotlib qt
set_browser_backend('qt')

from eeg_flow.tasks.ica_comp_select import(
    prep_ica_selection,
)

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [ ]:
"""
PARTICIPANT: str (for example "P03", "P28")
GROUP:       str ["G1", "G2", "G3", "G4", "G5", "G6", "G7", "G8"]
TASK:        str, ["oddball", "UT"]
RUN:         int [1,2]
"""

PARTICIPANT = "P02"
GROUP       = "G2"
TASK        = "oddball"
RUN         = 1

In [ ]:
ica1, ica2, DERIVATIVES_SUBFOLDER, FNAME_STEM, EXPERIMENTER, raw_ica_fit1, raw, locks = prep_ica_selection(PARTICIPANT, GROUP, TASK, RUN)
DERIVATIVES_ICA = DERIVATIVES_SUBFOLDER / "plots" / "ica"

## 4.1 Annotate bad ICs from ICA1 for mastoids
 - At this stage, let's only focus on the mastoids. Look for:
 - heartbeat in the IC-time series
 - muscle/noise on the mastoids on the topographic map

In [ ]:
figs_ica_sources_mastoids = ica1.plot_sources(
    title=f"{FNAME_STEM} | ICA1 sources Mastoids | {EXPERIMENTER}",
    show=True,
    inst=raw_ica_fit1,
)

In [ ]:
%matplotlib inline
# to avoid the bug
plt.figure(figsize=(2,2))
plt.plot([])


In [ ]:
## might need to restart twice, make sure all pages are displayed, and not just 1
%matplotlib qt
figs_ica_comp_mastoids = ica1.plot_components(
    title=f"{FNAME_STEM} | ICA1 components Mastoids | {EXPERIMENTER}",
    show=True,
    inst=raw_ica_fit1,
)

In [ ]:
figs_ica_comp_mastoids

In [ ]:
ica_folder = DERIVATIVES_SUBFOLDER / "plots" / "ica"
timestampStr = datetime.now().strftime("%Y-%m-%d__%H-%M")
for i, fig in enumerate(figs_ica_comp_mastoids):
    FNAME_ICA1_COMP = (
        DERIVATIVES_ICA / f"allComponents_ICA1_{EXPERIMENTER}_{i+1}_timestampStr.svg"
    )
    fig.savefig(FNAME_ICA1_COMP, transparent=True)

## Save derivatives ICA1

The ICA decomposition can be saved.

In [ ]:
FNAME_ICA1 = DERIVATIVES_SUBFOLDER / f"{FNAME_STEM}_step4_reviewed-1st-{EXPERIMENTER}-ica.fif"
ica1.save(FNAME_ICA1, overwrite=False)

In [ ]:
#%% Clean the other channels
# The first step is to prepare the raw object for an ICA, and for suggestions
# from ICLabel. The steps are very similar to the previous ones.
del raw_ica_fit1
raw.drop_channels(["M1", "M2"])

In [ ]:
# filter
raw_ica_fit2 = raw
raw_ica_fit2.filter(
    l_freq=1.0,
    h_freq=100.0,  # Note the higher frequency
    picks="eeg",
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)

In [ ]:
# change the reference to a common average reference (CAR)
raw_ica_fit2.set_montage(None)
raw_ica_fit2.add_reference_channels(ref_channels="CPz")
raw_ica_fit2.set_montage("standard_1020")
raw_ica_fit2.set_eeg_reference("average", projection=False)
# Note that the CAR is excluding the bad channels.

## 4.2 Annotate bad ICs from ICA2 for EEG
 - At this stage, let's only focus on the mastoids. Look for:
 - heartbeat in the IC-time series
 - muscle/noise on the mastoids on the topographic map


In [ ]:
# Visual inspection
figs_ica_sources = ica2.plot_sources(
    title=f"{FNAME_STEM} | ICA2 sources | {EXPERIMENTER}",
    show=True, 
    inst=raw_ica_fit2
)

In [ ]:
%matplotlib inline
# to avoid the bug
plt.figure(figsize=(2,2))
plt.plot([])


In [ ]:
# same here, this might require to be run twice
%matplotlib qt
figs_ica_comp = ica2.plot_components(
    title=f"{FNAME_STEM} | ICA2 components | {EXPERIMENTER}",
    show=True,
    inst=raw_ica_fit2
)

In [ ]:
timestampStr = datetime.now().strftime("%Y-%m-%d__%H-%M")
for i, fig in enumerate(figs_ica_comp):
    FNAME_ICA2_COMP = (
        DERIVATIVES_ICA / f"allComponents_ICA2_{EXPERIMENTER}_{i+1}_timestampStr.svg"
    )
    fig.savefig(FNAME_ICA2_COMP, transparent=True)

## 4.3 Save derivatives

The ICA decomposition can be saved.

In [ ]:
FNAME_ICA2 = DERIVATIVES_SUBFOLDER / f"{FNAME_STEM}_step4_reviewed-2nd-{EXPERIMENTER}-ica.fif"
ica2.save(FNAME_ICA2, overwrite=False)

Regardless of the success of the task, the locks must be released.
If this step is forgotten, someone might have to remove the corresponding `.lock` file manually.

In [ ]:
for lock in locks:
    lock.release()
del locks  # delete would release anyway

Output of step 4
- Reviewed ICA 1 (_reviewed-1st-{EXPERIMENTER}-ica.fif)
- Reviewed ICA 2 (_reviewed-2nd-{EXPERIMENTER}-ica.fif)